In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from csv import reader
import csv
import re
import pathlib
import sys
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
from cove import MTLSTM
from transformers import AutoTokenizer, BertModel
from sklearn.metrics import f1_score

In [2]:
# Bert Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

MAX_LEN_GC = 350
MAX_LEN_B = 256

def tokenizerfnc(str):
    return tokenizer.encode(str,max_length=MAX_LEN_B, pad_to_max_length=True)

In [3]:
class CNN_GC(nn.Module):

    def __init__(self,out_dim=200,gc_dim=900,max_len=MAX_LEN_GC, dropout=0.5):
        super(CNN_GC, self).__init__()
        
        """cove_model = CoVeEmbeddings(
            word_embeddings_dir='../model/text/stanford/glove/', 
            tokenizer=tokenizer,
            max_sequence_length=max_len, verbose=20)
        glove_model = GloVeEmbeddings()
        glove_model.load_model(dest_dir='../model/text/stanford/glove/', process=False)"""
        
        self.gc_dim = gc_dim
        self.out_dim = out_dim
        self.max_len = max_len
        self.dropout = dropout

        self.conv_3 = nn.Conv1d(gc_dim, out_dim, 3, stride=1, padding=1)
        self.conv_5 = nn.Conv1d(gc_dim, out_dim, 5, stride=1, padding=2)
        self.conv_7 = nn.Conv1d(gc_dim, out_dim, 7, stride=1, padding=3)
        self.attn = nn.Linear(3*out_dim*max_len, max_len)

    def forward(self,x):
        #cove_embed = cove_model.encode(x)
        #tokens = [sentence.split(" ") for sentence in x]
        #glove_embed = glove_model.encode(tokens)
        #x = torch.cat([cove_embed,glove_embed], 2)
        
        conv_3 = F.relu(self.conv_3(x))
        conv_5 = F.relu(self.conv_5(x))
        conv_7 = F.relu(self.conv_7(x))
        x = torch.cat([conv_3,conv_5,conv_7], 1)
        #print(x.shape)
        
        non_linear_x = F.relu(x.view(-1, 600*self.max_len))
        #print(non_linear_x.shape)
        attn_weights = F.softmax(self.attn(non_linear_x), dim=1)
        #print(attn_weights.shape)
        #attn_applied = torch.bmm(attn_weights.unsqueeze(1), x)
        #attn_applied = attn_weights*x
        attn_applied = torch.zeros(x.shape[0], x.shape[1], x.shape[2])
        for i in range(x.shape[0]):
            attn_applied[i,:,:] = x[i,:,:]*attn_weights[i]
        #print("hello")
        #print(attn_applied.shape)
        
        x = attn_applied.sum(dim=2)
        return x
    
class CNN_BERT(nn.Module):

    def __init__(self, out_dim=100, embed_dim=1024, max_len=MAX_LEN_B, dropout=0.5):
        super(CNN_BERT, self).__init__()
        
        self.embed_dim = embed_dim
        self.out_dim = out_dim # not 100?
        self.max_len = max_len # ?
        self.dropout = dropout

        self.bert_layer = BertModel.from_pretrained('bert-large-uncased')
        self.conv_3 = nn.Conv1d(1, out_dim, embed_dim * 3, stride=embed_dim)
        self.conv_4 = nn.Conv1d(1, out_dim, embed_dim * 4, stride=embed_dim)
        self.conv_5 = nn.Conv1d(1, out_dim, embed_dim * 5, stride=embed_dim)

    def get_conv_out(self,conv,x, num):
        return F.max_pool1d(F.relu(conv(x)), 
                            self.max_len - num + 1).view(-1, self.out_dim)

    def forward(self,x):
        x = self.bert_layer(x)[0].view(-1, 1, self.embed_dim * self.max_len)
        conv_3 = self.get_conv_out(self.conv_3, x, 3)
        conv_4 = self.get_conv_out(self.conv_4, x, 4)
        conv_5 = self.get_conv_out(self.conv_5, x, 5)
        x = torch.cat([conv_3, conv_4, conv_5], 1)
        x = F.dropout(x, p=self.dropout)
        return x

In [4]:
######################################################################
#
#   Architecture is as follows
#   1. Bert word encoding + CNN with max pooling for sent.embed
#   2. Glove & CoVe word Encoding  + CNN with attention for sent.embed
#   3. (1) and (2) concated and fed to MLP for classification (1-sugg)

class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.l0 = nn.Linear(900,300) # BERT-> CNN (300), G,C->CNN (600)
        self.l1 = nn.Linear(300,300)
        self.l3 = nn.Linear(300,2)

    def forward(self,x):
        # print(x.shape)
        # print(x.type())
        x = torch.sigmoid(self.l0(x))
        # print(x.shape)
        x = torch.sigmoid(self.l1(x))
        # print(x.shape)
        x = torch.sigmoid(self.l3(x))
        # print(x.shape)
        return x


class SuggestionClassifier(nn.Module):
    def __init__(self):
        # path all the models together
        # BERT->CNN + CNN->ATT
        super(SuggestionClassifier,self).__init__()
        self.CNN_b = CNN_BERT()
        self.CNN_gc = CNN_GC()
        self.MLP = MLP()


    def forward(self,x, y):
        #print("=========================")
        bert_x = self.CNN_b(x)
        #print("Total Model", bert_x.shape)
        gc_x = self.CNN_gc(y)
        #print("Total Model", gc_x.shape)
        out = torch.cat((bert_x,gc_x), 1) # TODO : ensure dimensions align (1x300, 1x600)
        #print("Total", out.shape)
        out = self.MLP(out) # Dim : 1x900
        #print("Total", out.shape)
        return out

In [5]:
#########################
# HYPER PARAMETER TUNING

NUM_EPOCHS = 1
WEIGHT_DECAY = 3
BATCH_SIZE = 32
NFOLDS = 1
NUM_WORKERS = 1
LEARNING_RATE = 0.1 # TODO : Experiment, as LR not given

########################
# MODEL CREATION

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight.data)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SuggestionClassifier()
# TODO:  WEIGHT INIT
model.apply(init_weights)
model.train()
model.to(device)

#######################
# OPTIMIZATION

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [6]:
#############################
# HELPER FUNCTIONS

def train(model, train_loader, valid_loader, test_loader, foldId, numEpochs,
           outputs_cove_with_glove, outputs_cove_with_glove_test):
    '''
    Args:

    Ret: 
    '''

    model.train()
    val_loss_min = None
    val_loss_min_delta = 0
    val_patience = 0
    val_loss_counter = 0

    # TODO : 10-fold cross validation (build 10 models)
    # TODO : Ensemble, pick top three models based on train loss
    # TODO : Voting Mechanism to evaluate on Validation data
    
    # for each fold
    #     for each epoch
    #       min_batch_train(model)
    #       val_loss = model(trial_data)
    #       early_stopping(val_loss)
    #     models += models
    #     three_models = select_best_3(models) based on train_loss
        
    for epoch in range(numEpochs):
        avg_loss = 0.0
        for batch_num, batch in enumerate(train_loader):
            glove_then_last_layer_cove = outputs_cove_with_glove(*batch.sentence)
            target = torch.zeros(glove_then_last_layer_cove.shape[0], MAX_LEN_GC, 900)
            max_sentence_len_in_batch = max(batch.sentence[1].tolist())
            target[:, :max_sentence_len_in_batch, :] = glove_then_last_layer_cove
            glove_then_last_layer_cove = target.permute(0,2,1)
            
            bert_enc = batch.bert_enc.to(device)
            glove_cove_enc = glove_then_last_layer_cove.to(device)
            
            optimizer.zero_grad()
            outputs = model(bert_enc, glove_cove_enc)
            #print(outputs.shape)
            
            labels = batch.label.long()
            labels = labels.to(device)
            loss = criterion(outputs, labels)
            #print(loss)
            loss.backward()
            optimizer.step()
            #print("Optimizer Complete")

            avg_loss += loss.item()
            #print(avg_loss)

            if batch_num % 50 == 49:
                print('Fold: {}\t Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(foldId+1, 
                                                                                 epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0

            torch.cuda.empty_cache()
            del bert_enc
            del glove_cove_enc
            del labels
            
            # break

        model_name = "jessi_A_fold_"+str(foldId+1)+"_epoch_"+str(epoch+1)+".pt"
        print("Created Model: ", model_name)
        torch.save(model.state_dict(), model_name)

        #val_loss, val_acc = validate(model, valid_loader, outputs_cove_with_glove_valid)
        test_loss, test_acc = validate(model, test_loader, outputs_cove_with_glove_test)
#         print('Epoch{:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}\tTest Loss: {:.4f}\tTest Accuracy: {:.4f}'.format(
#             epoch, val_loss, val_acc, test_loss, test_acc))
        print('Epoch{:.4f}\tTest Loss: {:.4f}\tTest Accuracy: {:.4f}'.format(
            epoch, test_loss, test_acc))

        # Early stopping
#         if val_loss_min is None:
#             val_loss_min = val_loss
#         elif val_loss_min >= (val_loss + val_loss_min_delta) :
#             val_loss_counter += 1
#             if (val_loss_counter > val_patience) :
#                 print("Validation Loss: {}\t Lowest Validation Loss {}\t".format(val_loss, val_loss_min))
#                 print("Training stopped early, Epoch :"+str(epoch))
#                 break
#         else:
#             val_loss_min = val_loss

    return test_acc, model_name


def validate(model, valid_loader, outputs_cove_with_glove):
    '''
    Args:

    Ret:
    '''

    model.eval()
    test_loss = []
    all_predictions = []
    all_labels = []
    accuracy = 0
    total = 0

    for batch_num, batch in enumerate(valid_loader):
        
        glove_then_last_layer_cove = outputs_cove_with_glove(*batch.sentence)
        target = torch.zeros(glove_then_last_layer_cove.shape[0], MAX_LEN_GC, 900)
        max_sentence_len_in_batch = max(batch.sentence[1].tolist())
        target[:, :max_sentence_len_in_batch, :] = glove_then_last_layer_cove
        glove_then_last_layer_cove = target.permute(0,2,1)

        bert_enc = batch.bert_enc.to(device)
        glove_cove_enc = glove_then_last_layer_cove.to(device)
        outputs = model(bert_enc, glove_cove_enc)

        labels = batch.label.long()
        #labels = labels.to(device)
    
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        #loss = criterion(outputs, labels)
        #accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        #total += len(labels)
        #test_loss.extend([loss.item()]*BATCH_SIZE)
        
        all_labels.extend(list(np.array(labels)))
        predictions = pred_labels.cpu()
        all_predictions.extend(list(np.array(predictions)))
        

        #torch.cuda.empty_cache()
        del bert_enc
        del glove_cove_enc
        del labels
    
        # break

    score = f1_score(all_labels, all_predictions)
    print(all_labels, all_predictions)
    model.train()
    return 0, score
    #return 0, accuracy/total



In [7]:
########################
# DATA LOADING

VALID_TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
VALID_LABEL = data.Field(sequential=False, use_vocab=False)
VALID_TEXT_BERT = data.Field(
    use_vocab=False,
    batch_first=True,
    pad_token=tokenizer.pad_token_id,
    tokenize=tokenizerfnc
)

# valid_dataset = SuggestionDataset(load_data(filename="SubtaskA_Trial_Test_Labeled.csv"))
valid_path = "C:\\Users\\thank\\Downloads\\CMU MCDS\\11747\\project\suggestionMining\\data\\Subtask-A\\SubtaskA_Trial_Test_Labeled.csv"
valid_dataset = data.TabularDataset(
            path=valid_path, format='csv',
            skip_header = False,
            fields={'sentence':[('sentence',VALID_TEXT),('bert_enc',VALID_TEXT_BERT)],
                    'label':('label',VALID_LABEL)
                    })

VALID_TEXT.build_vocab(valid_dataset, vectors=GloVe(name='840B', dim=300, cache='.embeddings'))
outputs_cove_with_glove_valid = MTLSTM(n_vocab=len(VALID_TEXT.vocab), vectors=VALID_TEXT.vocab.vectors, residual_embeddings=True, model_cache='.embeddings')

valid_iter = data.Iterator(
        (valid_dataset),
        batch_size=BATCH_SIZE)

In [8]:
# data = load_data()
# data_folds = create_folds(data)
model_perfs = []
model_names = []

train_path = "C:\\Users\\thank\\Downloads\\CMU MCDS\\11747\\project\suggestionMining\\data\\Subtask-A\\train_{}.csv"
test_path = "C:\\Users\\thank\\Downloads\\CMU MCDS\\11747\\project\suggestionMining\\data\\Subtask-A\\test_{}.csv"

for i in range(NFOLDS):
    
    print("FOLD: ", i)
    
    TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
    LABEL = data.Field(sequential=False, use_vocab=False)
    TEXT_BERT = data.Field(
        use_vocab=False,
        batch_first=True,
        pad_token=tokenizer.pad_token_id,
        tokenize=tokenizerfnc
    )

    # train loader
    train_dataset = data.TabularDataset(
            path=train_path.format(i), format='csv',
            skip_header = False,
            fields={'sentence':[('sentence',TEXT),('bert_enc',TEXT_BERT)],
                    'label':('label',LABEL)
                    })

    TEXT.build_vocab(train_dataset, vectors=GloVe(name='840B', dim=300, cache='.embeddings'))
    outputs_cove_with_glove = MTLSTM(n_vocab=len(TEXT.vocab), vectors=TEXT.vocab.vectors, 
                                     residual_embeddings=True, model_cache='.embeddings')

    train_iter = data.Iterator(
        (train_dataset),
        batch_size=BATCH_SIZE)
    
    print("Created Train Iterator")
    
    TEST_TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
    TEST_LABEL = data.Field(sequential=False, use_vocab=False)
    TEST_TEXT_BERT = data.Field(
        use_vocab=False,
        batch_first=True,
        pad_token=tokenizer.pad_token_id,
        tokenize=tokenizerfnc
    )
    
    # test loader
    test_dataset = data.TabularDataset(
            path=test_path.format(i), format='csv',
            skip_header = False,
            fields={'sentence':[('sentence',TEST_TEXT),('bert_enc',TEST_TEXT_BERT)],
                    'label':('label',TEST_LABEL)
                    })

    TEST_TEXT.build_vocab(test_dataset, vectors=GloVe(name='840B', dim=300, cache='.embeddings'))
    outputs_cove_with_glove_test = MTLSTM(n_vocab=len(TEST_TEXT.vocab), vectors=TEST_TEXT.vocab.vectors, 
                                          residual_embeddings=True, model_cache='.embeddings')

    test_iter = data.Iterator(
        (test_dataset),
        batch_size=BATCH_SIZE)
    
    print("Created Test Iterator")
    
    test_loss, model_name = train(model, train_iter, valid_iter, test_iter, i, NUM_EPOCHS, 
                                  outputs_cove_with_glove, outputs_cove_with_glove_test)

    model_perfs.append(test_loss)
    model_names.append(model_name)


print(model_perfs)
print(model_names)
best_three_model_idx = np.flip(np.argsort(model_perfs))[0:3]
models = []
for idx in best_three_model_idx:
    # load the model
    #temp_model = SuggestionClassifier()
    temp_model = SuggestionClassifier()
    temp_model.load_state_dict(torch.load(model_names[idx]))
    models.append(temp_model)
# testing
print(len(models))

FOLD:  0
Created Train Iterator
Created Test Iterator
Created Model:  jessi_A_fold_1_epoch_1.pt
Epoch0.0000	Test Loss: 0.0000	Test Accuracy: 0.0000
FOLD:  1
Created Train Iterator
Created Test Iterator
Created Model:  jessi_A_fold_2_epoch_1.pt
Epoch0.0000	Test Loss: 0.0000	Test Accuracy: 0.0000
[0.0, 0.0]
['jessi_A_fold_1_epoch_1.pt', 'jessi_A_fold_2_epoch_1.pt']
2


In [9]:
def test(models, test_loader, outputs_cove_with_glove, id_map=0):
    '''
    Args:

    Ret:
    '''

    test_loss = []
    accuracy = 0
    total = 0
    
    all_labels = []
    all_predictions = []
    outcomes = []
    
    for batch_num, batch in enumerate(test_loader):

        glove_then_last_layer_cove = outputs_cove_with_glove(*batch.sentence)
        target = torch.zeros(glove_then_last_layer_cove.shape[0], MAX_LEN_GC, 900)
        max_sentence_len_in_batch = max(batch.sentence[1].tolist())
        target[:, :max_sentence_len_in_batch, :] = glove_then_last_layer_cove
        glove_then_last_layer_cove = target.permute(0,2,1)

        bert_enc = batch.bert_enc.to(device)
        glove_cove_enc = glove_then_last_layer_cove.to(device)
        
        predictions = torch.tensor(np.zeros(len(batch.label)))

        for model in models:
            model.eval()
            outputs = model(bert_enc, glove_cove_enc)
            
            _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
            # print(np.array(pred_labels.view(-1).cpu()))
            predictions += pred_labels.view(-1).cpu()
            model.train()

        predictions = torch.round(predictions/3)
        
        labels = batch.label.long()
        
        accuracy += torch.sum(torch.eq(predictions, labels))
        total += len(labels)
        
        all_labels.extend(list(np.array(labels)))
        
        predictions = np.array(predictions)
        all_predictions.extend(list(predictions))
        
        outcomes += [ ['ID','SENT',predictions[i]] for i in range(len(predictions)) ]
        
        del bert_enc
        del glove_cove_enc
        del labels
        
        break
    
    f = open("result.csv", "w", newline="")
    writer = csv.writer(f)
    writer.writerows(outcomes)
    f.close()
    
    score = f1_score(all_labels, all_predictions)
    # print(all_labels, all_predictions)
    print("F1 Score:", str(score))
    
    #return np.mean(test_loss), accuracy/total
    return accuracy/total, score


TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
LABEL = data.Field(sequential=False, use_vocab=False)
TEXT_BERT = data.Field(
    use_vocab=False,
    batch_first=True,
    pad_token=tokenizer.pad_token_id,
    tokenize=tokenizerfnc
)

test_path = "C:\\Users\\thank\\Downloads\\CMU MCDS\\11747\\project\suggestionMining\\data\\Subtask-A\\SubtaskA_Trial_Test_Labeled.csv"
test_dataset = data.TabularDataset(
            path=test_path, format='csv',
            skip_header = False,
            fields={'sentence':[('sentence',TEXT),('bert_enc',TEXT_BERT)],
                    'label':('label',LABEL)
                    })


TEXT.build_vocab(test_dataset, vectors=GloVe(name='840B', dim=300, cache='.embeddings'))
outputs_cove_with_glove = MTLSTM(n_vocab=len(TEXT.vocab), vectors=TEXT.vocab.vectors, residual_embeddings=True, model_cache='.embeddings')


test_iter = data.Iterator(
        (test_dataset),
        batch_size=BATCH_SIZE)

test(models, test_iter, outputs_cove_with_glove)

F1 Score: 0.5714285714285715


(0.4, 0.5714285714285715)

In [10]:
# i = 1
# train_path = "C:\\Users\\thank\\Downloads\\CMU MCDS\\11747\\project\suggestionMining\\data\\Subtask-A\\train_{}.csv"
# test_path = "C:\\Users\\thank\\Downloads\\CMU MCDS\\11747\\project\suggestionMining\\data\\Subtask-A\\test_{}.csv"

# TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
# LABEL = data.Field(sequential=False, use_vocab=False)
# TEXT_BERT = data.Field(
#     use_vocab=False,
#     batch_first=True,
#     pad_token=tokenizer.pad_token_id,
#     tokenize=tokenizerfnc
# )

# # train loader
# train_dataset = data.TabularDataset(
#         path=train_path.format(i), format='csv',
#         skip_header = False,
#         fields={'sentence':[('sentence',TEXT),('bert_enc',TEXT_BERT)],
#                 'label':('label',LABEL)
#                 })

# TEXT.build_vocab(train_dataset, vectors=GloVe(name='840B', dim=300, cache='.embeddings'))
# outputs_cove_with_glove = MTLSTM(n_vocab=len(TEXT.vocab), vectors=TEXT.vocab.vectors, residual_embeddings=True, model_cache='.embeddings')

# train_iter = data.Iterator(
#     (train_dataset),
#     batch_size=5)


# print("Created Train Iterator")

# TEST_TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
# TEST_LABEL = data.Field(sequential=False, use_vocab=False)
# TEST_TEXT_BERT = data.Field(
#     use_vocab=False,
#     batch_first=True,
#     pad_token=tokenizer.pad_token_id,
#     tokenize=tokenizerfnc
# )

# # test loader
# test_dataset = data.TabularDataset(
#         path=test_path.format(i), format='csv',
#         skip_header = False,
#         fields={'sentence':[('sentence',TEST_TEXT),('bert_enc',TEST_TEXT_BERT)],
#                 'label':('label',TEST_LABEL)
#                 })

# TEST_TEXT.build_vocab(test_dataset, vectors=GloVe(name='840B', dim=300, cache='.embeddings'))
# outputs_cove_with_glove_test = MTLSTM(n_vocab=len(TEST_TEXT.vocab), vectors=TEST_TEXT.vocab.vectors, residual_embeddings=True, model_cache='.embeddings')

# test_iter = data.Iterator(
#     (test_dataset),
#     batch_size=5)

# print("Created Test Iterator")

# for batch_num, batch in enumerate(train_iter):
#     avg_loss = 0.0
#     glove_then_last_layer_cove = outputs_cove_with_glove(*batch.sentence)
#     target = torch.zeros(5, 512, 900)
#     max_sentence_len_in_batch = max(batch.sentence[1].tolist())
#     target[:, :max_sentence_len_in_batch, :] = glove_then_last_layer_cove
#     glove_then_last_layer_cove = target.permute(0,2,1)

#     bert_enc = batch.bert_enc.to(device)
#     glove_cove_enc = glove_then_last_layer_cove.to(device)

#     optimizer.zero_grad()
#     outputs = model(bert_enc, glove_cove_enc)
#     print(outputs.shape)
#     target = batch.label.reshape(5,1)
#     print(target.shape)

#     print(outputs, target)
#     loss = criterion(outputs, batch.label.long())
#     print(loss)
#     loss.backward()
#     optimizer.step()
#     print("Optimizer Complete")

#     avg_loss += loss.item()
#     print(avg_loss)

#     if batch_num % 50 == 49:
#         print('Fold: {}\t Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(foldId+1, epoch+1, batch_num+1, avg_loss/50))
#         avg_loss = 0.0

#     torch.cuda.empty_cache()
#     del bert_enc
#     del glove_cove_enc

#     break


# models = []
# temp_model = SuggestionClassifier()
# temp_model.load_state_dict(torch.load('jessi_A_fold_1_epoch_1.pt'))
# models.append(temp_model)
# temp_model = SuggestionClassifier()
# temp_model.load_state_dict(torch.load('jessi_A_fold_2_epoch_1.pt'))
# models.append(temp_model)